In [29]:
import sqlite3
import pandas as pd
import numpy as np
import datetime

In [16]:
people_person = pd.read_csv('drive-download-20180422T230536Z-001/people_person.csv')

In [39]:
pet_pets = pd.read_csv('drive-download-20180422T230536Z-001/pets_pet.csv')

In [17]:
people_person.head(3)

,id,first_name,last_name,email,channel,date_joined,photo,fee,gender
0,1,Meaghan,Lapete,meaghan.lapete820@aol.com,NaN,2015-07-12 15:13:16.673652,https://placekitten.com/360/208,0.0,f
1,2,Aracelis,Luhr,aracelis.luhr844@rover.com,NaN,2015-07-12 13:44:21.949482,https://placekitten.com/392/227,0.0,f
2,3,Vernita,Amodei,vernita.amodei542@rover.com,NaN,2015-07-12 11:36:06.992714,https://placekitten.com/275/300,0.0,f


In [19]:
people_person.dtypes

id               int64
first_name      object
last_name       object
email           object
channel         object
date_joined     object
photo           object
fee            float64
gender          object
dtype: object

In [20]:
#converting the date joined column to a datetime object and not a string 
people_person['date_joined'] = pd.to_datetime(people_person['date_joined'])

# 1. How many users have signed up?

In [58]:
people_person.shape

(64416, 10)

### This specific platform has 64,416 users.

# 2. How many users signed up prior to 2017-01-12?

In [59]:
#Create a specific date column, we dont necesarily care about the timestamp.
people_person['date'] = people_person['date_joined'].dt.date

In [60]:
#Spot check to make sure we isolated the date.
people_person.head(2)

,id,first_name,last_name,email,channel,date_joined,photo,fee,gender,date
0,1,Meaghan,Lapete,meaghan.lapete820@aol.com,NaN,2015-07-12 15:13:16.673652,https://placekitten.com/360/208,0.0,f,2015-07-12
1,2,Aracelis,Luhr,aracelis.luhr844@rover.com,NaN,2015-07-12 13:44:21.949482,https://placekitten.com/392/227,0.0,f,2015-07-12


In [61]:
#Next step is to sort by the date signed up.
people_person.sort_values(by='date').head(2)

,id,first_name,last_name,email,channel,date_joined,photo,fee,gender,date
0,1,Meaghan,Lapete,meaghan.lapete820@aol.com,NaN,2015-07-12 15:13:16.673652,https://placekitten.com/360/208,0.0,f,2015-07-12
72,73,Napoleon,Quaresma,napoleon.quaresma561@gmail.com,NaN,2015-07-12 14:07:05.476956,https://placekitten.com/223/294,0.0,m,2015-07-12


In [62]:
#Need users prior to 2017-01-12

early_birds = people_person[(people_person['date'] < datetime.date(2017,1,12))]
early_birds.shape

(35500, 10)

### 35,500 signed up prior to 2017-01-12

# 3. What percentage of users have added pets?


In [63]:
#Looks like we are going to have to create a dataframe off of people_person and pets_pet, user_id is going to be the primary key.

In [48]:
# Query was writen in SQLite Studio and exported to csv
# This is the query below.
'''
SELECT a.id, b.owner_id
FROM people_person a
JOIN pets_pet b
ON (a.id = b.owner_id);
'''

'\nSELECT a.id, b.owner_id\nFROM people_person a\nJOIN pets_pet b\nON (a.id = b.owner_id);\n'

In [64]:
pet_owners = pd.read_csv('pet_owners.csv')
pet_owners.head(6)

,id,owner_id
0,12601,12601
1,12601,12601
2,12602,12602
3,12602,12602
4,12603,12603
5,12603,12603


In [65]:
pet_owners.shape

(77512, 2)

In [66]:
# The unique owners are in the id column, owner_id refers to the pets.
# By counting the unique ids we can arrive at the number of pet owners
num_pet_owners = len(pet_owners.id.unique())
num_pet_owners

51816

In [67]:
# count of users is the number of rows in the people_persons dataframe
users = people_person.shape[0]
users

64416

In [68]:
#
percentage = (num_pet_owners / users) * 100
percentage

80.43964232488823

# 4. Of those users, how many pets have they added on average?

In [70]:
# First we need to find the unique number of pets in pets dataframe.
pet_pets.head(2)

,id,name,description,gender,weight,birthday,plays_cats,plays_children,plays_dogs,spayed_neutered,house_trained,size,owner_id
0,219067,Dorla,Etiam risus.,f,77,2015-04-20,0,1,1,1,1,large,12601
1,219068,Magan,Massa ipsum felis.,f,85,2016-01-03,0,0,1,0,1,large,12601


In [72]:
count_pets = len(pet_pets.id)
count_pets

77512

In [73]:
#We also know the amount of pet owners
num_pet_owners

51816

In [75]:
count_pets, num_pet_owners

(77512, 51816)

In [76]:
pets_per_owner = (count_pets / num_pet_owners)
pets_per_owner

1.4959085996603365

# 5. What percentage of pets play well with cats?

In [77]:
# We know from the pet_pets dataframe whether or not a pet gets along with cats.

In [78]:
pet_pets.head(2)

,id,name,description,gender,weight,birthday,plays_cats,plays_children,plays_dogs,spayed_neutered,house_trained,size,owner_id
0,219067,Dorla,Etiam risus.,f,77,2015-04-20,0,1,1,1,1,large,12601
1,219068,Magan,Massa ipsum felis.,f,85,2016-01-03,0,0,1,0,1,large,12601


In [81]:
pet_pets.plays_cats.value_counts()

0    58308
1    19204
Name: plays_cats, dtype: int64

In [83]:
cat_friendly = (19204 / count_pets) * 100
cat_friendly

24.775518629373515